# Projeto 1 - Ciência dos Dados

Nome: Guilherme Martins

Nome: Giovana Cassoni

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [99]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [100]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\guilh\OneDrive\Área de Trabalho\Nova pasta (2)\cd\Projeto1_C.Dados-1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [101]:
filename = 'Air Fryer.xlsx'

In [102]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,air fryer pra canhoto,1
1,@vanessapeitos @daizenbvc o franky conseguiria...,0
2,adoro poder aplicar meus conhecimentos de enge...,1
3,"@leilodecornohu1 ""amigo estamos falando de buc...",0
4,se eu n mandar isso aqui na dm de alguma esqui...,0


In [103]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,@poetatulio @tatyene_mendes isso é uma xota ou...,0
1,"comprei uma air fryer p minha mãe, o bagulho caro",0
2,usando air fryer como se não houvesse o amanhã,1
3,minha mãe comprou uma air fryer e já ama mais ...,1
4,repita comigo: churrasco na air fryer não existe!,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [104]:
import re 

def cleanup(text):

    punctuation = '[!-.:?;]'

    pattern = re.compile(punctuation)

    text_subbed = re.sub(pattern, '', text)

    # barra = '[/\]'

    # padrao = re.compile(barra)

    # texto_novo = re.sub(padrao, ' ', text_subbed)

    return text_subbed

In [105]:
train = train.Treinamento.apply(str)
train = train.to_list()
train

['air fryer pra canhoto',
 '@vanessapeitos @daizenbvc o franky conseguiria colocar uma fritadeira air fryer no pau',
 'adoro poder aplicar meus conhecimentos de engenharia eletrica (acabei de arrumar a air fryer)',
 '@leilodecornohu1 "amigo estamos falando de buceta, não de uma air fryer " kkkkkkk',
 'se eu n mandar isso aqui na dm de alguma esquisita ainda hoje eu sou uma air fryer https://t.co/f2fjtqqhbi',
 '@nataliabridi viu que a mesma mulher colocava ate chocolate na 🐸 ate ele derreter pro amado poder descer? hahahahaha\né a buceta air fryer',
 'fui fazer um bolinho na air fryer, esborrou\nganhei bolinho de biscoitinhos de chocolate',
 'descobri que batata doce na air fryer é ruim',
 '@renataabritto a mulher era uma air fryer, uma air fryer',
 'eu queria uma air fryer',
 'a fritadeira elétrica nell fit é um eletroportátil que não pode faltar na sua cozinha. o produto proporciona uma alimentação mais saudável, pois não utiliza óleo/air fryer em seu processo de cozimento. #lojasmso 

In [106]:
lista_dos_tweets = []

lista_tweet = []

for tweet in train:

    lista_tweet.append(tweet)

    lista_dos_tweets.append(lista_tweet)

    lista_tweet = []

lista_dos_tweets

[['air fryer pra canhoto'],
 ['@vanessapeitos @daizenbvc o franky conseguiria colocar uma fritadeira air fryer no pau'],
 ['adoro poder aplicar meus conhecimentos de engenharia eletrica (acabei de arrumar a air fryer)'],
 ['@leilodecornohu1 "amigo estamos falando de buceta, não de uma air fryer " kkkkkkk'],
 ['se eu n mandar isso aqui na dm de alguma esquisita ainda hoje eu sou uma air fryer https://t.co/f2fjtqqhbi'],
 ['@nataliabridi viu que a mesma mulher colocava ate chocolate na 🐸 ate ele derreter pro amado poder descer? hahahahaha\né a buceta air fryer'],
 ['fui fazer um bolinho na air fryer, esborrou\nganhei bolinho de biscoitinhos de chocolate'],
 ['descobri que batata doce na air fryer é ruim'],
 ['@renataabritto a mulher era uma air fryer, uma air fryer'],
 ['eu queria uma air fryer'],
 ['a fritadeira elétrica nell fit é um eletroportátil que não pode faltar na sua cozinha. o produto proporciona uma alimentação mais saudável, pois não utiliza óleo/air fryer em seu processo de 

In [107]:
string = " "
for i in train:
    string = string + i + ' '
string = string.lower()
string = string.replace('air fryer', 'PRODUTO')
string = cleanup(string)
string.split()

['PRODUTO',
 'pra',
 'canhoto',
 '@vanessapeitos',
 '@daizenbvc',
 'o',
 'franky',
 'conseguiria',
 'colocar',
 'uma',
 'fritadeira',
 'PRODUTO',
 'no',
 'pau',
 'adoro',
 'poder',
 'aplicar',
 'meus',
 'conhecimentos',
 'de',
 'engenharia',
 'eletrica',
 'acabei',
 'de',
 'arrumar',
 'a',
 'PRODUTO',
 '@leilodecornohu1',
 'amigo',
 'estamos',
 'falando',
 'de',
 'buceta',
 'não',
 'de',
 'uma',
 'PRODUTO',
 'kkkkkkk',
 'se',
 'eu',
 'n',
 'mandar',
 'isso',
 'aqui',
 'na',
 'dm',
 'de',
 'alguma',
 'esquisita',
 'ainda',
 'hoje',
 'eu',
 'sou',
 'uma',
 'PRODUTO',
 'https//tco/f2fjtqqhbi',
 '@nataliabridi',
 'viu',
 'que',
 'a',
 'mesma',
 'mulher',
 'colocava',
 'ate',
 'chocolate',
 'na',
 '🐸',
 'ate',
 'ele',
 'derreter',
 'pro',
 'amado',
 'poder',
 'descer',
 'hahahahaha',
 'é',
 'a',
 'buceta',
 'PRODUTO',
 'fui',
 'fazer',
 'um',
 'bolinho',
 'na',
 'PRODUTO',
 'esborrou',
 'ganhei',
 'bolinho',
 'de',
 'biscoitinhos',
 'de',
 'chocolate',
 'descobri',
 'que',
 'batata',
 'doce

In [108]:
palavras = string.split()
serie_tweets = pd.Series(palavras)
serie_tweets.head(10)

0           PRODUTO
1               pra
2           canhoto
3    @vanessapeitos
4        @daizenbvc
5                 o
6            franky
7       conseguiria
8           colocar
9               uma
dtype: object

In [109]:
tabela_tweets = serie_tweets.value_counts()
tabela_tweets

PRODUTO       297
de            165
uma           129
e             111
a             106
             ... 
chatinho        1
sobrecarga      1
ah              1
química         1
gráfico         1
Length: 1589, dtype: int64

In [110]:
tabela_tweets_relativa = serie_tweets.value_counts(True)
tabela_tweets_relativa

PRODUTO       0.062910
de            0.034950
uma           0.027325
e             0.023512
a             0.022453
                ...   
chatinho      0.000212
sobrecarga    0.000212
ah            0.000212
química       0.000212
gráfico       0.000212
Length: 1589, dtype: float64

In [111]:
tabela_tweets_relativa.sum()

1.0

In [112]:
relevante = ['bom','empolgado','amo','feliz','comprei','comprou','animado','felicidade','mãe','compra']
não_relevante = ['parece','calor','ruim','queimo','queimar','quente']

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**